<a href="https://colab.research.google.com/github/MaratSaidov/textaugmentation/blob/master/arinaruck/quora%20insincerity%20classification/fairseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sacremoses subword_nmt
!pip install fairseq
!pip install fastBPE
!pip install tqdm
!pip install sentencepiece

     |████████████████████████████████| 870kB 32.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=7d40966c2a86d5200d263fa8158785bffea1e5ae8f75cad546471e358702e037
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 307kB 34.0MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2015588 sha256=641ca2e4002653dafaed73a15139c559b6c94497221730a806ed189e6eadfc98
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp36-cp36m-linux_x86_64.whl size=480849 sha256=c8a22aa08e00a4ab77ad3de2015b75392dcd886d52b68082399482a9aa9535e9
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
Successfully built f

In [2]:
import torch
import fairseq
import gc
import pandas as pd

from tqdm import tqdm_notebook

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'transformer.wmt16.en-de', ... ]

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


['bart.large',
 'bart.large.cnn',
 'bart.large.mnli',
 'bpe',
 'camembert.v0',
 'conv.stories',
 'conv.stories.pretrained',
 'conv.wmt14.en-de',
 'conv.wmt14.en-fr',
 'conv.wmt17.en-de',
 'data.stories',
 'roberta.base',
 'roberta.large',
 'roberta.large.mnli',
 'roberta.large.wsc',
 'tokenizer',
 'transformer.wmt14.en-fr',
 'transformer.wmt16.en-de',
 'transformer.wmt18.en-de',
 'transformer.wmt19.de-en',
 'transformer.wmt19.de-en.single_model',
 'transformer.wmt19.en-de',
 'transformer.wmt19.en-de.single_model',
 'transformer.wmt19.en-ru',
 'transformer.wmt19.en-ru.single_model',
 'transformer.wmt19.ru-en',
 'transformer.wmt19.ru-en.single_model',
 'transformer_lm.gbw.adaptive_huge',
 'transformer_lm.wiki103.adaptive',
 'transformer_lm.wmt19.de',
 'transformer_lm.wmt19.en',
 'transformer_lm.wmt19.ru',
 'xlmr.base.v0',
 'xlmr.large.v0']

In [3]:
# Load a transformer trained on WMT'16 En-De
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')
de2en.eval()  # disable dropout

# The underlying model is available under the *models* attribute
assert isinstance(de2en.models[0], fairseq.models.transformer.TransformerModel)

# Move model to GPU for faster translation
de2en.cuda()

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 2992273886/2992273886 [01:15<00:00, 39472305.22B/s]


loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.de-en.joined-dict.single_model.tar.gz from cache at /root/.cache/torch/pytorch_fairseq/f42bb1b72d293668a5c50d9589fd2f3cc27322e390b1ef4cf3fdcf625c0d2fd7.bf6e22453272c2cba218a5ccecd045f73e926c34c1d66c47c9b31233343820a9
extracting archive file /root/.cache/torch/pytorch_fairseq/f42bb1b72d293668a5c50d9589fd2f3cc27322e390b1ef4cf3fdcf625c0d2fd7.bf6e22453272c2cba218a5ccecd045f73e926c34c1d66c47c9b31233343820a9 to temp dir /tmp/tmpa3eoxxzq
| [de] dictionary: 42024 types
| [en] dictionary: 42024 types
Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_wmt_en_de_big', attention_dropout=0.1, bpe='fastbpe', bpe_codes='/root/.cache/torch/pytorch_fairseq/f42bb1b72d293668a5c50d9589fd2f3cc27322e390b1ef4cf3fdcf625c0d2fd7.bf6e22453272c2cba218a5ccecd045f73e926c34c1d66c47c9b31233343820a9/bpec

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (embed_tokens): Embedding(42024, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=8192, bias=True)
            (fc2): Linear(in_features=8192, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=T

In [0]:
# Batched translation
de2en.translate(['Hallo Welt!', 'Die Katze saß auf der Matte.'])
['Hello world!', 'The cat sat on the mat.']

['Hello world!', 'The cat sat on the mat.']

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!head /content/drive/My\ Drive/de_beam_5/de_0.txt

▁Wann ▁werden ▁Männer ▁und ▁Jung en ▁die ▁Wahrheit ▁lernen ▁und ▁ihre n ▁eigene n ▁Weg ▁gehen ?
▁Wann ▁lernen ▁Männer ▁und ▁Jung en ▁die ▁Wahrheit ▁und ▁gehen ▁ihre n ▁eigene n ▁Weg ?
▁Wann ▁werden ▁Männer ▁und ▁Jung en ▁die ▁Wahrheit ▁er lernen ▁und ▁ihre n ▁eigene n ▁Weg ▁gehen ?
▁Wann ▁werden ▁Männer ▁und ▁Jung en ▁die ▁Wahrheit ▁lernen ▁und ▁ihre ▁eigene n ▁Wege ▁gehen ?
▁Wann ▁werden ▁die ▁Männer ▁und ▁Jung en ▁die ▁Wahrheit ▁lernen ▁und ▁ihre n ▁eigene n ▁Weg ▁gehen ?
▁Warum ▁spricht ▁K er al ites ▁( In di en ) ▁nicht ▁Hin di ?
▁Warum ▁spricht ▁K er al ites ▁( In di en ) ▁Hin di ▁nicht ?
▁Warum ▁will ▁K er al ites ▁( In di en ) ▁Hin di ▁nicht ▁sprechen ?
▁Warum ▁spricht ▁K er al ites ▁( In di en ) ▁nicht ▁gerne ▁Hin di ?
▁Warum ▁spricht ▁K er al ites ▁( In di en ) ▁Hin di ▁nicht ▁gerne ?


In [0]:
import sentencepiece as spm
import sys

sp = spm.SentencePieceProcessor()
sp.Load("/content/drive/My Drive/sentencepiece.model")
input_name = '/content/drive/My Drive/de_beam_5/de'
output_name = '/content/drive/My Drive/de_beam_5_decoded/de'
file_num = 1000
for i in range(5, file_num):
      f1 = open(output_name + '_' + str(i) + '.txt', "w")
      f2 = open(input_name + '_' + str(i) + '.txt', "r")
      for line in f2.readlines():         
          f1.write(sp.DecodePieces(line.split(' ')))
      f1.close()
      f2.close()    
print("decoded")

decoded


In [0]:
!( find /content/drive/My\ Drive/de_beam_5/ -name '*.txt' -print0 | xargs -0 cat ) | wc -l
!( find /content/drive/My\ Drive/de_beam_5_decoded/ -name '*.txt' -print0 | xargs -0 cat ) | wc -l

4571400
4571400


In [0]:
!head /content/drive/My\ Drive/de_beam_5_decoded/de_0.txt

Wann werden Männer und Jungen die Wahrheit lernen und ihren eigenen Weg gehen?
Wann lernen Männer und Jungen die Wahrheit und gehen ihren eigenen Weg?
Wann werden Männer und Jungen die Wahrheit erlernen und ihren eigenen Weg gehen?
Wann werden Männer und Jungen die Wahrheit lernen und ihre eigenen Wege gehen?
Wann werden die Männer und Jungen die Wahrheit lernen und ihren eigenen Weg gehen?
Warum spricht Keralites (Indien) nicht Hindi?
Warum spricht Keralites (Indien) Hindi nicht?
Warum will Keralites (Indien) Hindi nicht sprechen?
Warum spricht Keralites (Indien) nicht gerne Hindi?
Warum spricht Keralites (Indien) Hindi nicht gerne?


In [0]:
!wc -l /content/drive/My\ Drive/de_beam_5_decoded/de_0.txt

4570 /content/drive/My Drive/de_beam_5_decoded/de_0.txt


In [0]:
input_name = '/content/drive/My Drive/de_beam_5_decoded/de'
output_name = '/content/drive/My Drive/fairseq/en'
for i in tqdm_notebook(1000):
  out_file = open(output_name + '_' + str(i) + '.txt', "w")
  in_file =  open(input_name + '_' + str(i) + '.txt', "r")
  lines_de = list(map(lambda x: x[:-1], in_file.readlines()[::5]))
  linecount = len(lines_de)
  for j in tqdm_notebook(range(linecount)):
      lines_en = de2en.translate(lines_de[j])
      out_file.write(lines_en.strip('"') + '\n')
  out_file.close()
  in_file.close()

In [0]:
output_file = '/content/drive/My Drive/fairseq/full_translation.txt'
in_file = '/content/drive/My Drive/fairseq/en'
out = open(output_file, 'w')
for i in range(1000):
    f = open(in_file + '_' + str(i) + '.txt', 'r')
    lines = f.readlines()
    out.writelines(lines)
    f.close()
out.close()

In [0]:
!( find /content/drive/My\ Drive/fairseq/ -name '*.txt' -print0 | xargs -0 cat ) | wc -l

1828560


In [0]:
!wc -l /content/drive/My\ Drive/fairseq/full_translation.txt

914280 /content/drive/My Drive/fairseq/full_translation.txt


In [7]:
bad_ones = []
for i in tqdm_notebook(range(999)):
  lines = pd.read_csv('/content/drive/My Drive/fairseq/en'  + '_' + str(i) + '.txt', sep="\n", header=None)
  if len(lines) != 914:
    bad_ones.append(i)
print(bad_ones)


[12, 32, 105, 106, 113, 145, 169, 172, 234, 247, 326, 335, 343, 379, 380, 383, 407, 418, 454, 457, 506, 521, 620, 637, 714, 719, 726, 748, 855, 889, 891, 902, 913, 934]


In [0]:
bad_ones = []
for i in range(1000):
  try:
    lines = pd.read_csv('/content/drive/My Drive/fairseq/en'  + '_' + str(i) + '.txt', sep="\n", header=None)
  except:
    bad_ones.append(i)

In [0]:
print(bad_ones)

[]


In [0]:
def get_rid_of_quotes(file):
  f = open(file, 'r')
  lines = f.readlines()
  f.close()
  f = open(file, 'w')
  for line in lines:
    f.write(line.strip('"'))
  f.close()

In [0]:
for i in bad_ones:
  get_rid_of_quotes('/content/drive/My Drive/fairseq/en' + '_' + str(i) + '.txt')